In [1]:
!pip install langchain openai langchain-openai

You should consider upgrading via the 'D:\Code\PromptEngineering_Langchain\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"), temperature=1.5)

In [4]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing by offering tools that automate tests and provide detailed reports on the results. It can also provide recommendations and suggestions for improving testing processes and addressing any issues that may arise during testing. Additionally, langsmith can offer test case management, issue tracking, and collaboration features that enhance overall efficiency and communication within the testing team. Overall, Langsmith aims to streamline the testing process and ensure that all components of the software are thoroughly examined for quality assurance.')

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [6]:
chain = prompt | llm 

In [7]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content="Langsmith can help with testing in several ways:\n\n1. Automated testing: Langsmith has the ability to generate test scripts and automate the execution of tests. This can speed up the testing process and improve test coverage.\n\n2. Synthetic testing: Langsmith can create synthetic data to simulate different testing scenarios. This can help uncover edge cases and ensure thorough testing.\n\n3. Load testing: Langsmith can also be used for load testing by simulating high levels of traffic on the system and monitoring performance metrics. This can help identify performance bottlenecks and ensure the system can handle expected traffic levels.\n\n4. Integration testing: By creating mock services or integrations with Langsmith, teams can perform end-to-end testing and verify the system behaves as expected when using different dependent components.\n\nOverall, Langsmith's versatility and adaptability make it a valuable tool for testing various aspects of a system and ensuri

In [8]:
# The output of a ChatModel (and therefore, of this chain) is a message. 
# However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [9]:
chain = prompt | llm | output_parser

In [10]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith is a powerful tool developed to assist with automating temporary data languages for building cohesive test environments. Testing can be time-consuming, tedious, and prone to human error, especially when manual efforts are required to set up the environment or generate test data. Langsmith simplifies these tasks by enabling testers to define and switch between various data languages that represent different scenarios, states, or configurations.\n\nUsing Langsmith can facilitate testing in the following ways:\n\n1. **Rapid Setup**: With Langsmith, testers can swiftly set up specific test data environments by simply selecting the intended data language, all in a programmatic and automated manner. This helps streamline the testing process and saves time by eliminating the need to manually configure environments.\n\n2. **Repeatability and Consistency**: Langsmith ensures that test configurations are consistent and reproducible across runs. Testers can establish distinct data lang

In [11]:
# Basic Example of a Prompt.
response = llm.invoke("What are the 7 wonders of the world?")
print(f"Response: {response}")

Response: content='The 7 wonders of the world are:\n\n1. Great Wall of China\n2. Petra, Jordan\n3. Christ the Redeemer, Brazil\n4. Machu Picchu, Peru\n5. Chichen Itza, Mexico\n6. Colosseum, Italy\n7. Taj Mahal, India'


In [12]:
# Basic Example of a Prompt Template.
prompt_template = ChatPromptTemplate.from_template(
    "List {n} cooking recipe ideas {cuisine} cuisine (name only)."
)
prompt = prompt_template.format(n=3, cuisine="italian")
print(f"Templated Prompt: {prompt}")

response = llm.invoke(prompt)
print(f"Response: {response}")

Templated Prompt: Human: List 3 cooking recipe ideas italian cuisine (name only).
Response: content='1. Spaghetti Carbonara\n2. Margherita Pizza\n3. Osso Buco'


# Response Parser
- formatting responses
- structured output

ex: Movie
- title: str
- genre: list[str]
- year: int

In [13]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Movie(BaseModel):
    title: str = Field(description="movie title")
    genre: list[str] = Field(description="movie genre")
    year: int = Field(description="year of relase")
    
parser = PydanticOutputParser(pydantic_object=Movie)

In [14]:

prompt_template_text = """
Response with a movie recommendation based on the query:\n
{format_instructions}\n
{query}
"""

format_instructions = parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"title": {"title": "Title", "description": "movie title", "type": "string"}, "genre": {"title": "Genre", "description": "movie genre", "type": "array", "items": {"type": "string"}}, "year": {"title": "Year", "description": "year of relase", "type": "integer"}}, "required": ["title", "genre", "year"]}
```


In [15]:
prompt_template = ChatPromptTemplate.from_template(
    prompt_template_text
)

prompt = prompt_template.format(query="A 90s movie with Nicolas Cage.", format_instructions=format_instructions)
text_output = llm.predict(prompt)
print(text_output)

d:\Code\PromptEngineering_Langchain\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{
    "title": "Face/Off",
    "genre": ["Action", "Crime", "Sci-Fi"],
    "year": 1997
}


In [16]:
parsed_output = parser.parse(str(text_output))
print(parsed_output)

title='Face/Off' genre=['Action', 'Crime', 'Sci-Fi'] year=1997


In [17]:
# Using LCEL
chain = prompt_template | llm | parser
response = chain.invoke({"query": "A 90s movie with Nicolas Cage.", "format_instructions": format_instructions})
print(response)

title='Face/Off' genre=['Action', 'Crime', 'Sci-Fi'] year=1997


# LLM Chains
 Chaining prompts, input of a prompt depends on the output of another prompt

In [18]:
prompt_1 = """
Come up with a short plot synopsis summary (2-3 lines) for a {genre} movie.
"""

prompt_2 = """
Suggest 5 movie title ideas for this movie: \n\n {synopsis}
"""

In [19]:
from langchain.chains import LLMChain, SequentialChain

p1_template = ChatPromptTemplate.from_template(prompt_1)
chain_1 = LLMChain(llm=llm, prompt=p1_template, output_key="synopsis")

p2_template = ChatPromptTemplate.from_template(prompt_2)
chain_2 = LLMChain(llm=llm, prompt=p2_template, output_key="titles")

In [20]:
complex_chain = SequentialChain(
    chains=[chain_1, chain_2],
    input_variables=["genre"],
    output_variables=["synopsis", "titles"],
    verbose=True,
)
output = complex_chain({"genre": "comedy"})
print(f"Output: {output}")

d:\Code\PromptEngineering_Langchain\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.
Output: {'genre': 'comedy', 'synopsis': "When two brothers inherit an eccentric circus from their late father, they must navigate zany performers, outrageous challenges, and sibling rivalry to keep the business afloat and save their family legacy. With explosive humor and heartwarming moments, they discover there's more to their inheritance than just a bigtop tent.", 'titles': '1. "Circus Chaos: The Battle of the Brothers"\n2. "Big Top Rivals: The Legacy of Laughs"\n3. "Taming the Tent: A Circus Comedy"\n4. "Sibling Spectacular: The Family Circus"\n5. "Legacy Under the Big Top: Brothers in Comedy"'}


# AGENT

In [22]:
!pip install numexpr

     ---------------------------------------- 96.6/96.6 KB 1.4 MB/s eta 0:00:00


You should consider upgrading via the 'D:\Code\PromptEngineering_Langchain\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [23]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.chains import LLMMathChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.chat_models.openai import ChatOpenAI
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor


llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

os.environ["SERPAPI_API_KEY"] = 

search = SerpAPIWrapper()
tools = [
    Tool(
        name="search",
        description="Search for information Google.",
        func=search.run,
    ),
    Tool(
        name="calculator",
        description="Use this tool to calculate the difference between numbers.",
        func=llm_math_chain.run,
    ),
]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])
agent_schema = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x["intermediate_steps"]),
}

agent = agent_schema | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is the population difference between 🇺🇸 and 🇬🇧?"})

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass `serpapi_api_key` as a named parameter. (type=value_error)